# case 1

In [3]:
from __future__ import annotations
import os
os.chdir('/home/lq/LQcode/2_project/PHMBench/PHMGA/')
print(f"Current working directory: {os.getcwd()}")
from langgraph.graph import StateGraph, END, START

from src.agents.dataset_preparer_agent import dataset_preparer_agent
from src.agents.execute_agent import execute_agent
from src.agents.inquirer_agent import inquirer_agent
from src.agents.plan_agent import plan_agent
from src.agents.reflect_agent import reflect_agent_node
from src.agents.report_agent import report_agent_node
from src.agents.shallow_ml_agent import shallow_ml_agent
from src.states.phm_states import PHMState
import yaml
import os
from src.utils import load_state, save_state



Current working directory: /home/lq/LQcode/2_project/PHMBench/PHMGA


/home/lq/.conda/envs/PA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:


config_path = "config/case1.yaml"
print(f"--- Loading configuration from {config_path} ---")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

state_save_path = config['state_save_path']
builder_cfg = config.get('builder', {})
min_depth = builder_cfg.get('min_depth', 0)
max_depth = builder_cfg.get('max_depth', float('inf'))

# --- Check for existing state ---
if os.path.exists(state_save_path):
    print(f"\n--- Found existing state file at {state_save_path}. Skipping builder workflow. ---")
    state = load_state(state_save_path)






--- Loading configuration from config/case1.yaml ---

--- Found existing state file at /home/lq/LQcode/2_project/PHMBench/PHMGA/save/case1/case1_built_state.pkl. Skipping builder workflow. ---

--- Loading state from /home/lq/LQcode/2_project/PHMBench/PHMGA/save/case1/case1_built_state.pkl ---
...done.
Successfully loaded state with 4 nodes.


## 1 inquiring

In [5]:
print("\nStep 1: Inquiring - Performing similarity analysis...")
inquirer_updates = inquirer_agent(state, metrics=["cosine", "euclidean"])
state = state.copy(update=inquirer_updates)
print(f"Output: Updated state with similarity_results.")


Step 1: Inquiring - Performing similarity analysis...
Output: Updated state with similarity_results.


/tmp/ipykernel_5013/2378391913.py:3: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=inquirer_updates)


## 2 data preparation

In [6]:
from src.utils import load_signal_data
metadata_path, h5_path, ref_ids,test_ids = config['metadata_path'], config['h5_path'], config['ref_ids'], config['test_ids']
ref_signals, ref_labels = load_signal_data(metadata_path, h5_path, ref_ids)
test_signals, test_labels = load_signal_data(metadata_path, h5_path, test_ids)


Loading data for IDs: [47050, 47044, 47047, 47053, 47056]
Loading data for IDs: [47051, 47045, 47048, 47054, 47057]


In [7]:
test_signals

{'47051': array([[[  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         ...,
         [  -0.44000003,  -19.2000008 ],
         [  -0.54000002,  -24.        ],
         [  -0.48000002,  -28.80000114]]], shape=(1, 1249999, 2)),
 '47045': array([[[  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         ...,
         [  -0.46000004,    9.60000038],
         [  -0.56000006,    2.4000001 ],
         [  -0.50000006,   -4.80000019]]], shape=(1, 1249999, 2)),
 '47048': array([[[  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         ...,
         [  -0.40000004,  -11.1999998 ],
         [  -0.42000002,  -17.6000004 ],
         [  -0.48000002,  -23.20000076]]], shape=(1, 1249999, 2)),
 '47054': array([[[  -2.6200001 , -102.4000015 ],
         [  -2.6200001 , -102.4000015 ],
         [  -2.6200001 ,

In [11]:
# 2. Dataset Preparer Agent: Prepare datasets for ML
print("\nStep 2: Preparing - Creating datasets for ML model...")
preparer_updates = dataset_preparer_agent(state)
state = state.copy(update=preparer_updates)
print(f"Output: Updated state with datasets.")





Step 2: Preparing - Creating datasets for ML model...
Output: Updated state with datasets.


/tmp/ipykernel_5013/1584315041.py:4: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=preparer_updates)


In [16]:
state.datasets['fft_01_ch1']['X_train'].shape, state.datasets['fft_01_ch1']['y_train'].shape, state.datasets['fft_01_ch1']['X_test'].shape, state.datasets['fft_01_ch1']['y_test'].shape


((5, 625000), (5,), (5, 625000), (5,))

## 3  train model

In [12]:
# 3. Shallow ML Agent: Train and evaluate a simple ML model
print("\nStep 3: Training - Running shallow ML model...")
# Note: shallow_ml_agent directly takes the datasets, not the full state
ml_updates = {"ml_results": shallow_ml_agent(datasets=state.datasets)}
state = state.copy(update=ml_updates)
print(f"Output: Updated state with ml_results.")




Step 3: Training - Running shallow ML model...


/home/lq/.conda/envs/PA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Output: Updated state with ml_results.


/home/lq/.conda/envs/PA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/tmp/ipykernel_5013/3994341950.py:5: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=ml_updates)


## 4 reporting

In [14]:
# 4. Report Agent: Generate the final report
print("\nStep 4: Reporting - Generating final analysis report...")
report_updates = report_agent_node(state)
state = state.copy(update=report_updates)
print("Output: Updated state with the final report.")


Step 4: Reporting - Generating final analysis report...

--- Report Agent LLM Response ---
```markdown
# Bearing Fault Diagnosis Report

## 流程概览

The diagnostic process, as illustrated by the DAG (`/home/lq/LQcode/2_project/PHMBench/PHMGA/save/case1/graphs/dag_20250804_161125.png`), consists of 6 nodes. The process begins with signal acquisition and preprocessing. This is followed by feature extraction using FFT on two channels: `ds_fft_01_ch1` and `ds_fft_02_ch2`. The extracted features are then fed into machine learning models for classification, ultimately aiming to identify potential bearing faults.

## 特征/相似度洞察

Similarity statistics are currently unavailable. A detailed analysis of channel similarities and reasons behind high/low scores is not possible without this data.

## 模型评估

The following table summarizes the performance metrics of the individual models and the ensemble model:

| model      |   accuracy |   f1 |   cv_accuracy |   cv_f1 |
|:-----------|-----------:|-----:|-

/tmp/ipykernel_5013/1608283778.py:4: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  state = state.copy(update=report_updates)
